In [10]:
from PIL import Image
from transformers import AutoTokenizer, AutoProcessor, AutoModelForImageTextToText

model_path = "nanonets/Nanonets-OCR-s"

model = AutoModelForImageTextToText.from_pretrained(
    model_path,
    torch_dtype="auto",
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_path)
processor = AutoProcessor.from_pretrained(model_path)



    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], padding=True, return_tensors="pt")
    inputs = inputs.to(model.device)

    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]

    output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return output_text[0]

image_path = "/path/to/your/document.jpg"
result = ocr_page_with_nanonets_s(image_path, model, processor, max_new_tokens=15000)
print(result)


AttributeError: 'dict' object has no attribute 'to_dict'

In [7]:
# scratch/test_extraction_item.py

from extraction_io.ExtractionItems import ExtractionItem

# 1) A valid single-item dict that we want to parse:
raw_item = {
    "field_name": "BorrowerName",
    "description": "Name of the borrower",
    "probable_pages": [1, 2],
    "type": "key-value",          # must match Literal exactly
    "multipage_value": False,
    "multiline_value": False,
    "extra_rules": {"regex": "\\w+"}
}

# 2) Parse/validate using Pydantic v2's `model_validate`:
item = ExtractionItem.model_validate(raw_item)

print("Parsed ExtractionItem successfully:")
print(f"  field_name       = {item.field_name!r}")
print(f"  description      = {item.description!r}")
print(f"  probable_pages   = {item.probable_pages!r}")
print(f"  type             = {item.type!r}")
print(f"  multipage_value  = {item.multipage_value!r}")
print(f"  multiline_value  = {item.multiline_value!r}")
print(f"  extra_rules      = {item.extra!r}")

Parsed ExtractionItem successfully:
  field_name       = 'BorrowerName'
  description      = 'Name of the borrower'
  probable_pages   = [1, 2]
  type             = 'key-value'
  multipage_value  = False
  multiline_value  = False
  extra_rules      = {}


In [2]:
# scratch/test_extraction_items.py

from extraction_io.ExtractionItems import ExtractionItem, ExtractionItems
from pydantic import ValidationError

# 1) Example list of two config items:
raw_config = [
    {
        "field_name": "BorrowerName",
        "description": "Name of the borrower",
        "probable_pages": [1],
        "type": "key-value",
        "multipage_value": False,
        "multiline_value": False,
        "extra_rules": {"regex": "\\w+"}
    },
    {
        "field_name": "benefits_list",
        "description": "List of benefits",
        # no probable_pages provided ⇒ defaults to []
        "type": "bullet-points",
        "multipage_value": False,
        "multiline_value": False,
        # omit extra_rules entirely, it will default to {}
    }
]

# 2) Attempt to parse the list as ExtractionItems:
try:
    config_model = ExtractionItems.model_validate(raw_config)
    print("Parsed ExtractionItems successfully. Contents:")
    for idx, item in enumerate(config_model):
        print(f"Item {idx}:")
        print(f"  field_name      = {item.field_name!r}")
        print(f"  description     = {item.description!r}")
        print(f"  probable_pages  = {item.probable_pages!r}")
        print(f"  type            = {item.type!r}")
        print(f"  multipage_value = {item.multipage_value!r}")
        print(f"  multiline_value = {item.multiline_value!r}")
        print(f"  extra_rules     = {item.extra_rules!r}")
        print()
except ValidationError as e:
    print("ValidationError while parsing ExtractionItems:")
    print(e)

Parsed ExtractionItems successfully. Contents:
Item 0:
  field_name      = 'BorrowerName'
  description     = 'Name of the borrower'
  probable_pages  = [1]
  type            = 'key-value'
  multipage_value = False
  multiline_value = False
  extra_rules     = {'regex': '\\w+'}

Item 1:
  field_name      = 'benefits_list'
  description     = 'List of benefits'
  probable_pages  = []
  type            = 'bullet-points'
  multipage_value = False
  multiline_value = False
  extra_rules     = {}

